In [ ]:
from ratdata import data_manager as dm, process, ingest, plot as rdplot
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import scipy.signal as signal

In [ ]:
dm.db_connect('rat_data.db')

In [ ]:
beta_low = 11
beta_high = 30
fontsize = 30
fontsize_ax = 22

In [ ]:
ylims = {
        "rat A4": (-1e-7, 5e-6),
        "rat B1": (-0.0001, 0.0002),
        "rat B2": (-0.000025, 0.000025),
        "rat B3": (-0.00005, 0.0001),
        "rat B5": (-0.00004, 0.00005),
        "rat C4": (-8e-5, 7e-5),
        "rat C6": (-0.00005, 0.00015),
        "rat D1": (-2.5e-5, 4e-5),
        "rat D2": (-0.000025, 0.00004),
        "rat D4": (-4e-5, 3e-5)
    }

ohda_mean_spectra = []
ohda_std_spectra = []
sham_mean_spectra = []
sham_std_spectra = []

for rat in dm.Rat.select():
    print(f"{rat.full_label}")
    spectra = []
    ff = None
    for r in dm.RecordingFile.select().where(
        (dm.RecordingFile.condition == 'baseline') & (dm.RecordingFile.rat == rat)):
            filename = Path(r.dirname) / r.filename
            print(f"  {r.filename}")
            rec = ingest.read_mce_matlab_file(filename)
            if dm.is_recording_rejected(filename.name):
                print("REJECTED")
                continue
            fs = int(1/rec.dt)
            x = np.mean(rec.electrode_data, 0)
            if dm.is_recording_sliced(r.filename):
                slice = dm.RecordingSlice.select().join(dm.RecordingFile).where(dm.RecordingFile.filename == filename.name).get()
                start = int(slice.start * fs)
                end = int((slice.start + slice.length) * fs)
                x = x[start:end]
            f, psd = signal.welch(x, fs, nperseg=int(2*fs))
            m, b = process.fit_oof(f, psd, 2, 100)
            f[0] = 1e-20
            oof = f ** m * (np.e ** b)
            detrended = psd - oof
            idx = np.logical_and(f >= 10, f <= 35)
            if ff is None:
                ff = f[idx]
            spectra.append(detrended[idx])
    
    avg = np.mean(np.array(spectra), 0)
    std = np.std(np.array(spectra), 0)
    if rat.group == "6OHDA":
        ohda_mean_spectra.append(avg)
        ohda_std_spectra.append(std)
    elif rat.group == "control":
        sham_mean_spectra.append(avg)
        sham_std_spectra.append(std)
    
    fig = plt.figure(figsize=(10, 6))
    plt.plot(ff, np.mean(np.array(spectra), 0), linewidth=5, color=rdplot.sham_ohda_palette[rat.full_label])
    plt.fill_between(ff, avg-std, avg+std, facecolor=rdplot.sham_ohda_palette[rat.full_label], alpha=0.4)
    plt.axhline(0, color='red')
    for s in spectra:
        plt.plot(ff, s, color=(0.5, 0.5, 0.5), linewidth=0.3)
        
    plt.ylim(ylims[rat.full_label])
    plt.xlim([10, 35])
    ax = fig.axes[0]
    ax.tick_params(axis='both', which='major', labelsize=fontsize_ax)
    ax.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
    ax.yaxis.offsetText.set_fontsize(fontsize_ax)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.title(f"{rat.full_label}", fontsize=fontsize)
    plt.xlabel("Frequency [Hz]", fontsize=fontsize_ax)
    
    plt.savefig(f"plots/peaks/averaged_{rat.full_label}.svg", bbox_inches="tight")
    plt.savefig(f"plots/peaks/averaged_vec_{rat.full_label}.png", bbox_inches="tight")

In [ ]:
avg = np.mean(np.array(ohda_mean_spectra), 0)
std = (np.sum(np.array(ohda_std_spectra) ** 2, 0)) ** 0.5 / (len(ohda_std_spectra) ** 0.5)

fig = plt.figure(figsize=(10, 6))
plt.plot(ff, avg, 0, linewidth=5, color=rdplot.sham_ohda_palette["ohda"])
plt.fill_between(ff, avg-std, avg+std, facecolor=rdplot.sham_ohda_palette["ohda"], alpha=0.4)
plt.axhline(0, color='red')

plt.ylim([-5e-5, 5e-5])
plt.xlim([10, 35])
ax = fig.axes[0]
ax.tick_params(axis='both', which='major', labelsize=fontsize_ax)
ax.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
ax.yaxis.offsetText.set_fontsize(fontsize_ax)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title(f"6-OHDA rats", fontsize=fontsize)
plt.xlabel("Frequency [Hz]", fontsize=fontsize_ax)

plt.savefig(f"plots/peaks/averaged_rat_ohda.svg", bbox_inches="tight")
plt.savefig(f"plots/peaks/averaged_vec_rat_ohda.png", bbox_inches="tight")

In [ ]:
avg = np.mean(np.array(sham_mean_spectra), 0)
std = (np.sum(np.array(sham_std_spectra) ** 2, 0)) ** 0.5 / (len(ohda_std_spectra) ** 0.5)

fig = plt.figure(figsize=(10, 6))
plt.plot(ff, avg, 0, linewidth=5, color=rdplot.sham_ohda_palette["sham"])
plt.fill_between(ff, avg-std, avg+std, facecolor=rdplot.sham_ohda_palette["sham"], alpha=0.4)
plt.axhline(0, color='red')

plt.ylim([-5e-5, 5e-5])
plt.xlim([10, 35])
ax = fig.axes[0]
ax.tick_params(axis='both', which='major', labelsize=fontsize_ax)
ax.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
ax.yaxis.offsetText.set_fontsize(fontsize_ax)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title(f"Sham operated rats", fontsize=fontsize)
plt.xlabel("Frequency [Hz]", fontsize=fontsize_ax)

plt.savefig(f"plots/peaks/averaged_rat_control.svg", bbox_inches="tight")
plt.savefig(f"plots/peaks/averaged_vec_rat_control.png", bbox_inches="tight")

In [ ]:
spectra = []
for rat in dm.Rat.select().where((dm.Rat.group == "control") & (dm.Rat.full_label != "rat A4")):
    print(f"{rat.full_label}")
    ff = None
    for r in dm.RecordingFile.select().where(
        (dm.RecordingFile.condition == 'baseline') & (dm.RecordingFile.rat == rat)):
            filename = Path(r.dirname) / r.filename
            print(f"  {r.filename}")
            rec = ingest.read_mce_matlab_file(filename)
            if dm.is_recording_rejected(filename.name):
                print("REJECTED")
                continue
            fs = int(1/rec.dt)
            x = np.mean(rec.electrode_data, 0)
            if dm.is_recording_sliced(r.filename):
                slice = dm.RecordingSlice.select().join(dm.RecordingFile).where(dm.RecordingFile.filename == filename.name).get()
                start = int(slice.start * fs)
                end = int((slice.start + slice.length) * fs)
                x = x[start:end]
            f, psd = signal.welch(x, fs, nperseg=int(2*fs))
            m, b = process.fit_oof(f, psd, 2, 100)
            f[0] = 1e-20
            oof = f ** m * (np.e ** b)
            detrended = psd - oof
            idx = np.logical_and(f >= 10, f <= 35)
            if ff is None:
                ff = f[idx]
            spectra.append(detrended[idx])
fig = plt.figure(figsize=(10, 6))
avg = np.mean(np.array(spectra), 0)
std = np.std(np.array(spectra), 0)
plt.plot(ff, np.mean(np.array(spectra), 0), linewidth=5, color=rdplot.sham_ohda_palette[rat.full_label])
plt.fill_between(ff, avg-std, avg+std, facecolor=rdplot.sham_ohda_palette[rat.full_label], alpha=0.4)
plt.axhline(0, color='red')

plt.ylim([-5e-5, 5e-5])
plt.xlim([10, 35])
ax = fig.axes[0]
ax.tick_params(axis='both', which='major', labelsize=fontsize_ax)
ax.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
ax.yaxis.offsetText.set_fontsize(fontsize_ax)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title(f"Sham operated rats", fontsize=fontsize)
plt.xlabel("Frequency [Hz]", fontsize=fontsize_ax)


plt.savefig(f"plots/peaks/averaged_control.svg", bbox_inches="tight")
plt.savefig(f"plots/peaks/averaged_vec_control.png", bbox_inches="tight")

In [ ]:
spectra = []
for rat in dm.Rat.select().where(dm.Rat.group == "6OHDA"):
    print(f"{rat.full_label}")
    ff = None
    for r in dm.RecordingFile.select().where(
        (dm.RecordingFile.condition == 'baseline') & (dm.RecordingFile.rat == rat)):
            filename = Path(r.dirname) / r.filename
            print(f"  {r.filename}")
            rec = ingest.read_mce_matlab_file(filename)
            if dm.is_recording_rejected(filename.name):
                print("REJECTED")
                continue
            fs = int(1/rec.dt)
            x = np.mean(rec.electrode_data, 0)
            if dm.is_recording_sliced(r.filename):
                slice = dm.RecordingSlice.select().join(dm.RecordingFile).where(dm.RecordingFile.filename == filename.name).get()
                start = int(slice.start * fs)
                end = int((slice.start + slice.length) * fs)
                x = x[start:end]
            f, psd = signal.welch(x, fs, nperseg=int(2*fs))
            m, b = process.fit_oof(f, psd, 2, 100)
            f[0] = 1e-20
            oof = f ** m * (np.e ** b)
            detrended = psd - oof
            idx = np.logical_and(f >= 10, f <= 35)
            if ff is None:
                ff = f[idx]
            spectra.append(detrended[idx])
fig = plt.figure(figsize=(10, 6))
avg = np.mean(np.array(spectra), 0)
std = np.std(np.array(spectra), 0)
plt.plot(ff, np.mean(np.array(spectra), 0), linewidth=5, color=rdplot.sham_ohda_palette[rat.full_label])
plt.fill_between(ff, avg-std, avg+std, facecolor=rdplot.sham_ohda_palette[rat.full_label], alpha=0.4)
plt.axhline(0, color='red')

plt.ylim([-5e-5, 5e-5])
plt.xlim([10, 35])
ax = fig.axes[0]
ax.tick_params(axis='both', which='major', labelsize=fontsize_ax)
ax.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
ax.yaxis.offsetText.set_fontsize(fontsize_ax)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title(f"6-OHDA rats", fontsize=fontsize)
plt.xlabel("Frequency [Hz]", fontsize=fontsize_ax)

plt.savefig(f"plots/peaks/averaged_ohda.svg", bbox_inches="tight")
plt.savefig(f"plots/peaks/averaged_vec_ohda.png", bbox_inches="tight")